You can use a JSON viewer to help visualizing a SPARQL query parsed: http://jsonviewer.stack.hu/

Documentation to parse SPARQL queries with RDFLib: https://rdflib.readthedocs.io/en/stable/_modules/rdflib/plugins/sparql/parser.html

In [1]:
import pandas as pd
import os

# print(os.system('pwd'))
# print(os.system('ls'))

df = pd.read_csv('/home/jovyan/work/graphsummary/sample.csv')


for index, row in df.iterrows():
    query = row['query']
    print(row['query'])
    print('\n')
    break

CONSTRUCT { ?s ?p ?o . } WHERE { SERVICE <http://bio2rdf.org/sparql> { ?s ?p ?o . } } LIMIT 1




In [7]:
from rdflib.plugins.sparql.parser import Query, UpdateUnit
from rdflib.plugins.sparql.processor import translateQuery
from rdflib import URIRef, Variable
# query_string = 'SELECT * WHERE { ?s ?p ?o . ?s a ?type. ?s ?p ?o  . FILTER(?type = <http://ohoho>) . } LIMIT 14'
# query_string = 'SELECT * WHERE { ?s ?p ?o . ?s a <https://schema.org/Drug> .} LIMIT 14'
# query_string="SELECT DISTINCT  ?o ?p2 WHERE{ ?s ?p ?o OPTIONAL{ <http://nonsensical.com/1/1583905344087> ?p2 ?o} }LIMIT   1000"

# query_string = 'SELECT ?var1  ?var1Label WHERE {  ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q4423781> . ?var1 <http://www.w3.org/2000/01/rdf-schema#label>  ?var1Label . FILTER (  ( (  LANG (  ?var1Label  )  =  "ru" ) ) ) .}'

# query_string = """SELECT ?var1  ?var1Label WHERE { 
#     graph ?g {  
#         ?var1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q4423781> .
#         ?var1 <http://www.w3.org/2000/01/rdf-schema#label>  ?var1Label .
#         FILTER (  ( (  LANG (  ?var1Label  )  =  "ru" ) ) ) .
#     }
# }
# """
query_string = """SELECT ?var1  ?var1Label WHERE { 
    ?var1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q4423781> .
    ?var1 <http://www.w3.org/2000/01/rdf-schema#label>  ?var1Label .
    FILTER (  ( (  LANG (  ?var1Label  )  =  "ru" ) ) ) .
}
"""

parsed_query = translateQuery(Query.parseString(query_string, parseAll=True))

# print(parsed_query.algebra.p.p.p.expr)

# print(parsed_query.algebra.p.p.p.triples)
print(parsed_query.algebra)

SelectQuery_SelectQuery_{'p': Project_{'p': Filter_{'expr': RelationalExpression_{'expr': Builtin_LANG_{'arg': rdflib.term.Variable('var1Label'), '_vars': {rdflib.term.Variable('var1Label')}}, 'op': '=', 'other': rdflib.term.Literal('ru'), '_vars': set()}, 'p': BGP_{'triples': [(rdflib.term.Variable('var1'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P31'), rdflib.term.URIRef('http://www.wikidata.org/entity/Q4423781')), (rdflib.term.Variable('var1'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Variable('var1Label'))], '_vars': {rdflib.term.Variable('var1Label'), rdflib.term.Variable('var1')}}, '_vars': {rdflib.term.Variable('var1Label'), rdflib.term.Variable('var1')}}, 'PV': [rdflib.term.Variable('var1'), rdflib.term.Variable('var1Label')], '_vars': {rdflib.term.Variable('var1Label'), rdflib.term.Variable('var1')}}, 'datasetClause': None, 'PV': [rdflib.term.Variable('var1'), rdflib.term.Variable('var1Label')], '_vars': {rdflib.term.Variabl

In [3]:
# We show here that the object to manipulate is different depending on the SPARQL query
#query_string = 'SELECT * WHERE { ?s ?p ?o . ?s a ?type . } LIMIT 14'
# query_string = 'SELECT * WHERE { ?s ?p ?o . ?s a <https://schema.org/Drug> .} LIMIT 14'

parsed_query = translateQuery(Query.parseString(query_string, parseAll=True))

# Iterate over the triples defined in the SPARQL query
for triple in parsed_query.algebra.p.p.p.triples:
    print(triple)
    if str(triple[1]) == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
        if type(triple[2]) == Variable:
            print('Type is a variable to search: ' + triple[2])
        if type(triple[2]) == URIRef:
            print('Type is defined in the query: ' + triple[2])
            

(rdflib.term.Variable('var1'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P31'), rdflib.term.URIRef('http://www.wikidata.org/entity/Q4423781'))
(rdflib.term.Variable('var1'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Variable('var1Label'))


In [4]:
from rdflib.plugins.sparql.parser import parseUpdate
from pprint import pprint

qs = '''DELETE {
  <> a <urn:type:A> , <urn:type:b> ;
    <urn:p:a> ?x .
} INSERT {
  <> <urn:p:a> "hello" .
} WHERE {
  <> <urn:p.a> ?x .
}'''

q = parseUpdate(qs)
pprint(q)

{'prologue': [([], {})],
 'request': [{'delete': {'quads': {'triples': [([rdflib.term.URIRef(''), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:type:A'), rdflib.term.URIRef(''), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:type:b'), rdflib.term.URIRef(''), rdflib.term.URIRef('urn:p:a'), rdflib.term.Variable('x')], {})]}},
              'insert': {'quads': {'triples': [([rdflib.term.URIRef(''), rdflib.term.URIRef('urn:p:a'), literal_{'string': rdflib.term.Literal('hello')}], {})]}},
              'where': {'part': [{'triples': [([rdflib.term.URIRef(''), PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': rdflib.term.URIRef('urn:p.a')}]}]}, rdflib.term.Variable('x')], {})]}]}}]}


In [9]:
from rdflib.plugins.sparql.parser import parseQuery


query_string = """SELECT ?var1  ?var1Label WHERE { 
    ?var1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q4423781> .
    ?var1 <http://www.w3.org/2000/01/rdf-schema#label>  ?var1Label .
    FILTER (  ( (  LANG (  ?var1Label  )  =  "ru" ) ) ) .
}
"""

query_string = 'SELECT * WHERE { ?s ?p ?o . ?s a ?type. ?s ?p ?o  . FILTER(?type = <http://ohoho>) . } LIMIT 14'

parsed_query = parseQuery(query_string)

# print(parsed_query.algebra.p.p.p.expr)

print(parsed_query)

[[], SelectQuery_{'where': GroupGraphPatternSub_{'part': [TriplesBlock_{'triples': [([rdflib.term.Variable('s'), rdflib.term.Variable('p'), rdflib.term.Variable('o')], {}), ([rdflib.term.Variable('s'), PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')}]}]}, rdflib.term.Variable('type')], {}), ([rdflib.term.Variable('s'), rdflib.term.Variable('p'), rdflib.term.Variable('o')], {})]}, Filter_{'expr': ConditionalOrExpression_{'expr': ConditionalAndExpression_{'expr': RelationalExpression_{'expr': AdditiveExpression_{'expr': MultiplicativeExpression_{'expr': rdflib.term.Variable('type')}}, 'op': '=', 'other': AdditiveExpression_{'expr': MultiplicativeExpression_{'expr': rdflib.term.URIRef('http://ohoho')}}}}}}]}, 'limitoffset': LimitOffsetClauses_{'limit': rdflib.term.Literal('14', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))}}]
